In [43]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import pygame
import sys

In [44]:
# Genrate Medications
meds_df = pd.DataFrame({
    'Medicine': [],
    'Lifespan': []
})
max_meds = 10
lifespan_mean = 25
lifespan_std = 10

LIFESPAN_MIN = 5

medicine = set(['{:04x}'.format(np.random.randint(0, 0xFFFF)) for _ in range(max_meds)])
# ensures there are max_meds unique values
while len(medicine) < max_meds: medicine.add('{:04x}'.format(np.random.randint(0, 0xFFFF)))
medicine = list(medicine)

lifespan = np.random.normal(lifespan_mean, lifespan_std, max_meds)
lifespan = np.round(np.maximum(lifespan, LIFESPAN_MIN))

meds_df['Medicine'] = medicine
meds_df['Lifespan'] = lifespan

meds_df.head()


Medicine  Lifespan
0     8ac7      31.0
1     9cbe      22.0
2     bc4e      25.0
3     3439      16.0
4     600a      34.0

In [45]:
# Generate Clients
client_count = 100
MAXLAT = 256
MAXLONG = 256
MAX_DOSAGE = 3

latitudes = np.random.randint(0, MAXLAT, client_count)
longitudes = np.random.randint(0, MAXLONG, client_count)

cs_df = pd.DataFrame({
    'Client': ['Client {:03d}'.format(i) for i in range(client_count)],
    'Lat': latitudes,
    'Long': longitudes,
    'Medicine': np.random.choice(medicine, client_count),
    'Dosage': np.random.randint(1, MAX_DOSAGE, client_count)
})
# cs_df.index.name = 'Client'
cs_df.head()

Client  Lat  Long Medicine  Dosage
0  Client 000   20    63     3439       2
1  Client 001  176    58     c6b7       1
2  Client 002  120    41     d1d5       2
3  Client 003  216   110     c6b7       1
4  Client 004  121   113     bc4e       1

In [46]:
# Generate Pharmacies
pharmacy_count = 10
pharmacy_names = [f'pharm{i}' for i in range(pharmacy_count)]

latitudes = np.random.randint(0, MAXLAT, pharmacy_count)
longitudes = np.random.randint(0, MAXLONG, pharmacy_count)

rx_df = pd.DataFrame({
    'Name': pharmacy_names,
    'Lat': latitudes,
    'Long': longitudes
})
rx_df

Name  Lat  Long
0  pharm0   89    53
1  pharm1  127     0
2  pharm2  252   182
3  pharm3  180    12
4  pharm4   22   165
5  pharm5  158   149
6  pharm6  153    26
7  pharm7  213   238
8  pharm8  108   127
9  pharm9  250   178

In [47]:
# Assign Customers to Pharmacies
# client are assigned to closest pharm

# distance between each client and each pharm
client_coords = cs_df[['Lat', 'Long']].values
pharm = np.column_stack((latitudes[:pharmacy_count], longitudes[:pharmacy_count]))

distances = np.sqrt(
    (
        (client_coords[:, np.newaxis, :] - pharm[np.newaxis, :, :]) ** 2
    ).sum(axis=2))


# Assign each customer to the closest pharmacy
closest_pharmacies = np.argmin(distances, axis=1)
cs_df['Assigned_Pharmacy'] = closest_pharmacies

cs_df.head()


Client  Lat  Long Medicine  Dosage  Assigned_Pharmacy
0  Client 000   20    63     3439       2                  0
1  Client 001  176    58     c6b7       1                  6
2  Client 002  120    41     d1d5       2                  0
3  Client 003  216   110     c6b7       1                  5
4  Client 004  121   113     bc4e       1                  8

In [63]:
# Sort by assigned pharmacy
total_cs_df = cs_df[['Assigned_Pharmacy','Medicine','Dosage']].groupby(['Assigned_Pharmacy','Medicine']).sum().reset_index().set_index('Assigned_Pharmacy')
total_cs_df.head()

Medicine  Dosage
Assigned_Pharmacy                 
0                     3439       4
0                     600a       1
0                     8ac7       2
0                     9cbe       4
0                     af19       1

In [64]:
merged_df = pd.merge(rx_df, total_cs_df, right_index=True, left_index=True, how='left')
merged_df.sample(5)

Name  Lat  Long Medicine  Dosage
0  pharm0   89    53     af19       1
0  pharm0   89    53     bc4e       5
0  pharm0   89    53     600a       1
7  pharm7  213   238     9cbe       3
3  pharm3  180    12     d1d5       1

In [65]:
# Assign each pharmacy a list of medications that are required
from collections import Counter

# use assigned_pharmacy in cs_df to get the pharmacy name, and then get the medicine
# turn that into a counter and then into a dataframe into demand
def calculate_demand(group):
    return Counter({medicine: group.loc[group['Medicine'] == medicine, 'Dosage'].sum() for medicine in group['Medicine'].unique()})

# Apply the function to group by pharmacy ('Name') and calculate demand
demand_agg = merged_df.groupby('Name').apply(calculate_demand)

# Assign the aggregated demand back to the original DataFrame
pharmacy_df = merged_df.drop_duplicates(subset='Name').set_index('Name')
pharmacy_df['Demand'] = demand_agg

# Display the result
pharmacy_df.reset_index()


Name  Lat  Long Medicine  Dosage  \
0  pharm0   89    53     3439       4   
1  pharm1  127     0     9cbe       4   
2  pharm2  252   182     600a       1   
3  pharm3  180    12     3439       2   
4  pharm4   22   165     3439       3   
5  pharm5  158   149     3439       2   
6  pharm6  153    26     600a       2   
7  pharm7  213   238     3439       1   
8  pharm8  108   127     3439       2   
9  pharm9  250   178     600a       2   

                                              Demand  
0  {'3439': 4, '600a': 1, '8ac7': 2, '9cbe': 4, '...  
1                             {'9cbe': 4, 'af19': 1}  
2                                        {'600a': 1}  
3  {'3439': 2, '600a': 1, 'af19': 1, 'bc4e': 2, '...  
4  {'3439': 3, '7597': 5, '8ac7': 1, '9cbe': 5, '...  
5  {'3439': 2, '600a': 8, '7597': 5, '9cbe': 3, '...  
6  {'600a': 2, '7597': 1, 'c6b7': 1, 'd1d5': 1, '...  
7  {'3439': 1, '8ac7': 2, '9cbe': 3, 'c6b7': 1, '...  
8  {'3439': 2, '600a': 6, '8ac7': 2, '9cbe': 3, '...  
9  {'600a': 2, '7597': 3, '8ac7': 1, 'af19': 1, '...